# Initial Model

### from playtime_distribution, we found 2-5, 6-25, 26+ as good initial cutoffs

In [27]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# needs python 2
# import graphlab as gl
# gl.canvas.set_target('ipynb')

import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS

import os, sys
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))
from src import EDA
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# TODO Maybe show playtime distribution picture here

In [29]:
steam_200k_df = EDA.load_200k()

In [37]:
def rank_playtime(time):
    if time <= 1:
        return 0
    if time > 1 and time <= 5:
        return 1
    if time > 6 and time <= 25:
        return 2
    return 3
steam_200k_df["playtime_rank"] = steam_200k_df['playtime'].map(lambda value: rank_playtime(value))

In [38]:
steam_200k_df.head(2)

,uid,game_name,purchase_action,playtime,playtime_rank
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0,0
1,151603712,The Elder Scrolls V Skyrim,play,273.0,3


In [14]:
# Should we use user-user or item-item?
print('Number of users: ', steam_200k_df['uid'].value_counts().size)
print('Number of games: ', steam_200k_df['game_name'].value_counts().size)
# Use the games since there are less games than users (good for steam!)

Number of users:  12393
Number of games:  5155


In [24]:
# Setup a SparkSession
spark = SparkSession.builder.getOrCreate()
# Convert a Pandas DF to a Spark DF
spark_df = spark.createDataFrame(steam_200k_df)
spark_df.count()

200000

In [25]:
train, test = spark_df.randomSplit([0.8, 0.2], seed=427471138)

In [39]:
als_model = ALS(
    itemCol='game_name',
    userCol='uid',
    ratingCol='playtime_rank',
    nonnegative=True,    
    regParam=0.1,
    rank=10) 

In [40]:
als_model.fit(train)

IllegalArgumentException: 'requirement failed: Column game_name must be of type NumericType but was actually of type StringType.'

In [ ]:
# can create a dictionary of strings to incrementing uid and map into new column
# or get new dataset